<a href="https://colab.research.google.com/github/Gregory-Kimaru-K/Fullstack_Engineer_Projects/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
# import dependencies
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

#device config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#hyper parameters
num_epochs = 4
batch_size = 4
learning_rate = 0.001

#dataset has PILImage images of range [0,1]
#We transform them to Tensors of normalized range [-1, 1]
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True,
                                      download = True, transform = transform)

testset = torchvision.datasets.CIFAR10(root = './data', train = False,
                                      transform = transform)

train_loader = torch.utils.data.DataLoader(dataset =trainset, batch_size = batch_size,
                                           shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset =testset, batch_size = batch_size,
                                           shuffle = True)
classes = ('plane','car', 'bird','cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

#implement the network
class  NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.conv1 = nn.Conv2d(3, 6, 5)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.fc1 = nn.Linear(16*5*5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1, 16 * 5 * 5)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

model = NeuralNetwork().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

n_total_steps = len(train_loader)
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    #origin shape: [14, 3, 32, 32] = 4, 3, 1024
    #input_layer: 3 input channels, 6 output channels, 5 kernel size
    images = images.to(device)
    labels = labels.to(device)

    #forward pass
    outputs = model(images)
    loss = criterion(outputs, labels)

    #backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1) % 2000 == 0:
      print (f"Epoch [{epoch+1} / {num_epochs}], Step [{i + 1} / {n_total_steps}], loss: {loss.item():.4f}")

print ("Finished Training")

with torch.no_grad():
  n_correct = 0
  n_samples = 0
  n_class_correct = [0 for i in range(10)]
  n_class_samples = [0 for i in range(10)]
  for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)

    #max returns (value, index)
    _, predicted = torch.max(outputs, 1)
    n_samples += labels.size(0)
    n_correct += (predicted == labels).sum().item()

    for i in range(batch_size):
      label = labels[i]
      pred = predicted[i]
      if (label == pred):
        n_class_correct[label] += 1
      n_class_samples[label] += 1

  acc = 100.0 * (n_class_correct[i] / n_class_samples[i])
  print (f"Accuracy of the network: {acc} %")

  for i in range(10):
    acc = 100.0 * n_class_correct[i] / n_class_samples[i]
    print (f"Accuracy of {classes[i]}: {acc} %")


Files already downloaded and verified
Epoch [1 / 4], Step [2000 / 12500], loss: 2.3064
Epoch [1 / 4], Step [4000 / 12500], loss: 2.2781
Epoch [1 / 4], Step [6000 / 12500], loss: 2.3050
Epoch [1 / 4], Step [8000 / 12500], loss: 2.2964
Epoch [1 / 4], Step [10000 / 12500], loss: 2.3427
Epoch [1 / 4], Step [12000 / 12500], loss: 2.2024
Epoch [2 / 4], Step [2000 / 12500], loss: 1.8170
Epoch [2 / 4], Step [4000 / 12500], loss: 1.8522
Epoch [2 / 4], Step [6000 / 12500], loss: 2.1135
Epoch [2 / 4], Step [8000 / 12500], loss: 1.7232
Epoch [2 / 4], Step [10000 / 12500], loss: 2.5724
Epoch [2 / 4], Step [12000 / 12500], loss: 2.1151
Epoch [3 / 4], Step [2000 / 12500], loss: 2.0428
Epoch [3 / 4], Step [4000 / 12500], loss: 1.7575
Epoch [3 / 4], Step [6000 / 12500], loss: 1.6494
Epoch [3 / 4], Step [8000 / 12500], loss: 2.4534
Epoch [3 / 4], Step [10000 / 12500], loss: 1.6607
Epoch [3 / 4], Step [12000 / 12500], loss: 1.0619
Epoch [4 / 4], Step [2000 / 12500], loss: 1.1112
Epoch [4 / 4], Step [4000